In [1]:
import torch

a = torch.arange(1*3*15*15).view(1,3,15,15)
# print(a)
# print(a[:, 1:2])
# print(torch.chunk(a, 4, dim=1)[0][:, :3, :])
# # b = torch.ones(3, 4, 4)
print(a[:, :, :, 0:5])
# # c = torch.cat([a, b[:, :3, :]], dim=1)
# # print(c)

tensor([[[[  0,   1,   2,   3,   4],
          [ 15,  16,  17,  18,  19],
          [ 30,  31,  32,  33,  34],
          [ 45,  46,  47,  48,  49],
          [ 60,  61,  62,  63,  64],
          [ 75,  76,  77,  78,  79],
          [ 90,  91,  92,  93,  94],
          [105, 106, 107, 108, 109],
          [120, 121, 122, 123, 124],
          [135, 136, 137, 138, 139],
          [150, 151, 152, 153, 154],
          [165, 166, 167, 168, 169],
          [180, 181, 182, 183, 184],
          [195, 196, 197, 198, 199],
          [210, 211, 212, 213, 214]],

         [[225, 226, 227, 228, 229],
          [240, 241, 242, 243, 244],
          [255, 256, 257, 258, 259],
          [270, 271, 272, 273, 274],
          [285, 286, 287, 288, 289],
          [300, 301, 302, 303, 304],
          [315, 316, 317, 318, 319],
          [330, 331, 332, 333, 334],
          [345, 346, 347, 348, 349],
          [360, 361, 362, 363, 364],
          [375, 376, 377, 378, 379],
          [390, 391, 392, 393, 394],

In [27]:

def ensure_divisibility(numerator, denominator):
    """Ensure that numerator is divisible by the denominator."""
    assert numerator % denominator == 0, '{} is not divisible by {}'.format(
        numerator, denominator)


def divide(numerator, denominator):
    """Ensure that numerator is divisible by the denominator and return
    the division value."""
    # ensure_divisibility(numerator, denominator)
    return numerator // denominator

def split_tensor_along_last_dim(tensor, num_partitions,
                                kernel_size,
                                contiguous_split_chunks=False):
    """Split a tensor along its last dimension.
    Arguments:
        tensor: input tensor.
        num_partitions: number of partitions to split the tensor
        contiguous_split_chunks: If True, make each chunk contiguous
                                 in memory.
    """
    # Get the size and dimension.
    last_dim = tensor.dim() - 1
    last_dim_size = divide(tensor.size()[last_dim], num_partitions)
    print(last_dim_size)
    # Split.
    tensor_list = torch.split(tensor, last_dim_size, dim=last_dim)
    tensor_custom = [torch.cat([tensor_list[i], tensor[:, :, :, ((i+1)*last_dim_size):((i+1)*last_dim_size)+kernel_size-1]], dim=last_dim) for i in range(num_partitions-1)]
    tensor_custom.append(tensor_list[-1])

    # Note: torch.split does not create contiguous tensors by default.
    if contiguous_split_chunks:
        return tuple(chunk.contiguous() for chunk in tensor_list)

    return tensor_custom


print(split_tensor_along_last_dim(a, 4, 2)[0])

3
tensor([[[[  0,   1,   2,   3],
          [ 15,  16,  17,  18],
          [ 30,  31,  32,  33],
          [ 45,  46,  47,  48],
          [ 60,  61,  62,  63],
          [ 75,  76,  77,  78],
          [ 90,  91,  92,  93],
          [105, 106, 107, 108],
          [120, 121, 122, 123],
          [135, 136, 137, 138],
          [150, 151, 152, 153],
          [165, 166, 167, 168],
          [180, 181, 182, 183],
          [195, 196, 197, 198],
          [210, 211, 212, 213]],

         [[225, 226, 227, 228],
          [240, 241, 242, 243],
          [255, 256, 257, 258],
          [270, 271, 272, 273],
          [285, 286, 287, 288],
          [300, 301, 302, 303],
          [315, 316, 317, 318],
          [330, 331, 332, 333],
          [345, 346, 347, 348],
          [360, 361, 362, 363],
          [375, 376, 377, 378],
          [390, 391, 392, 393],
          [405, 406, 407, 408],
          [420, 421, 422, 423],
          [435, 436, 437, 438]],

         [[450, 451, 452, 453],
  

In [6]:
import torch
import torch.multiprocessing as mp
from model.resnet import ParallelConv2d
from model.initialize import initialize_model_parallel
import torch.distributed as dist



def test_conv(rank, ngpus_per_node, serial_conv, parallel_conv, input_data):
    if rank():
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)
    start.record()

    end.record()

    torch.cuda.synchronize()

    dist.init_process_group(backend="nccl", init_method="tcp://127.0.0.1:6006", world_size=ngpus_per_node, rank=rank)
    initialize_model_parallel()
    for p in parallel_conv.parameters():
        print("[ rank {} ] parallel weight : ".format(str(rank)), p)
    torch.cuda.set_device(rank)

    parallel_conv.cuda(rank)
    if rank == 0:
        serial_conv = torch.nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                                bias=False).cuda(rank)
        for p in serial_conv.parameters():
            print("serial weight :",  p)

    parallel_result = parallel_conv(parallel_test_data)

    if rank == 0:
        print("[Device : {} ]".format(str(torch.cuda.current_device())))
        serial_result = serial_conv(parallel_test_data)
        print("Parallel Result Shape : ", parallel_result.size())
        print("Serial Result Shape : ", serial_result.size())
        print("Parallel : ", parallel_result)
        print("Serial : ", serial_result)
        # parallel_result and serial_result should be the same
        assert torch.allclose(parallel_result, serial_result), "Parallel and Serial results are not the same"
        print("Conv layer Test Passed!!")




def main():
    # 한 노드의 GPU수를 가져옴.
    ngpus_per_node = torch.cuda.device_count()
    input_data = torch.randn(1, 3, 224, 224)
    # multiprocessing_distributed 변수가 true라면, world_size를 총 GPU개수로 설정한 후에, 메인 워커를 실행함.
    serial_conv = torch.nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                                bias=False)
    parallel_conv = ParallelConv2d(3, 64, kernel_size=7, stride=2, padding=3,
                                bias=False)
    serial_conv.weight = torch.ones((2,), dtype=torch.int8)
    serial_conv.weight.new_tensor(parallel_conv.weight)
    print("Are they same? : ", torch.allclose(serial_conv.weight, parallel_conv.weight))

    #mp.spawn(test_conv, nprocs=ngpus_per_node, args=(ngpus_per_node, serial_conv, parallel_conv, input_data))



if __name__ == '__main__':
    main()

TypeError: __init__() got an unexpected keyword argument 'device'

In [17]:
serial_conv = torch.nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                                bias=False)
serial_conv1 = torch.nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                                bias=False)


serial_conv.weight = serial_conv1.weight

print(torch.allclose(serial_conv.weight, serial_conv1.weight))
parallel_conv = torch.empty_like(serial_conv.weight).copy_(serial_conv.weight)

print(torch.allclose(serial_conv.weight, serial_conv1.weight))
print(serial_conv.weight)

False
False
Parameter containing:
tensor([[[[ 0.0753,  0.0312,  0.0084,  ...,  0.0498,  0.0002,  0.0824],
          [ 0.0799, -0.0630, -0.0130,  ...,  0.0809,  0.0636,  0.0688],
          [-0.0088, -0.0663,  0.0739,  ..., -0.0795, -0.0810,  0.0557],
          ...,
          [-0.0821, -0.0409, -0.0492,  ...,  0.0804,  0.0298,  0.0719],
          [-0.0641, -0.0250,  0.0629,  ..., -0.0809,  0.0558, -0.0650],
          [-0.0066,  0.0691,  0.0639,  ...,  0.0469,  0.0375,  0.0584]],

         [[-0.0543,  0.0693, -0.0768,  ..., -0.0251,  0.0316,  0.0338],
          [-0.0789, -0.0232, -0.0717,  ..., -0.0712,  0.0341, -0.0531],
          [ 0.0211, -0.0073, -0.0616,  ..., -0.0237,  0.0784,  0.0252],
          ...,
          [-0.0634, -0.0019, -0.0616,  ...,  0.0016, -0.0012,  0.0531],
          [-0.0738, -0.0161, -0.0709,  ...,  0.0176,  0.0616,  0.0707],
          [-0.0629,  0.0606,  0.0364,  ..., -0.0052,  0.0097, -0.0705]],

         [[-0.0606,  0.0284, -0.0641,  ..., -0.0329, -0.0711, -0.048

In [ ]:
import torch.nn.functional as F
import torch

t4d = torch.empty(3, 3, 4, 2) #쉽게 이미지를 생각하자 (batch, channel, height, width)
p1d = (1, 1) # pad last dim by 1 on each side, width의 왼쪽 오른쪽에 pad 추가
out = F.pad(t4d, p1d, "constant", 0) # effectively zero padding, 
print(out.size()) torch.Size([3, 3, 4, 4]) # 따라서 마지막 차원인 width가 2 증가.
p2d = (1, 1, 2, 2) # pad last dim by (1, 1) and 2nd to last by (2, 2),
#width 왼쪽 오른쪽에 pad 1개씩 추가, height 위 아래에 pad 2개씩 추가.
out = F.pad(t4d, p2d, "constant", 0) 
print(out.size()) torch.Size([3, 3, 8, 4])  
# height에 총 4개의 pad가 들어갔으므로 8, width에 2개가 들어가서 4가 되었다.
t4d = torch.empty(3, 3, 4, 2) 
p3d = (0, 1, 2, 1, 3, 3) # pad by (0, 1), (2, 1), and (3, 3) 
#마지막으로 (3,3) 은 channel 방향 앞 뒤로 3개씩 pad가 들어갔다.
out = F.pad(t4d, p3d, "constant", 0) 
print(out.size()) torch.Size([3, 9, 7, 3])
#다른건 보지 않고 channel의 차원(dimension 1)을 보면 6이 증가해서 9가 된 것을 볼 수 있다.